### Implement From structured  Output

In [2]:
student_response = "pass"

In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableBranch, RunnableSequence
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq


groq_api_key=os.getenv("GROQ_API_KEY")

model=ChatGroq(groq_api_key=groq_api_key,model_name="qwen-2.5-32b")

In [5]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain_core.output_parsers import PydanticOutputParser

In [6]:
class StuOutputCorrectness(BaseModel):
    sentiment: Literal['correct', 'partially_correct', 'incorrect'] = Field(description='Give the sentiment from the student response')
    
parser = PydanticOutputParser(pydantic_object=StuOutputCorrectness)

In [7]:
prompt = PromptTemplate(
    template="Classify the sentiment of the following student response text into correct, partially_correct or incorrect\n{response} \n{format_instruction}",
    input_variables=['response'],
    partial_variables={
        'format_instruction': parser.get_format_instructions()
    }
)

In [8]:
classifier_chain = prompt | model | parser

In [9]:
response = classifier_chain.invoke(
    {
        'response': "The sum of 2+2=5.0"
    }
)
response

StuOutputCorrectness(sentiment='incorrect')

In [10]:
response.sentiment

'incorrect'

In [11]:
classifier_chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
      +----------+       
      | ChatGroq |       
      +----------+       
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
+----------------------+ 
| StuOutputCorrectness | 
+----------------------+ 


## *Build Conditional Branch*

In [20]:
prompt_correct = PromptTemplate(
    template="Student give the correct response and the response is {response}. Now give a adaptive prompt so model can generate only best response for next step.",
    input_variables=['response']
)

prompt_partially_correct = PromptTemplate(
    template="""
    Student give the response which is partially correct and the response is {response}. Find where student did mistake and help him to understand his mistake. And tell his correctness and encourage him to overcome is mistake. Also tell how to think and to get correct response or ask student do you need more hints?. Now generate only the best adaptive prompt so model can provide the best response for next step.
    """,
    input_variables=['response'],
)

prompt_incorrect = PromptTemplate(
    template="""
    Student give totally incorrect response and the response is {response}. Analysis the student response and find out the student problem like student need more clarification about this topis. like why he give this incorrect response or give more elaborated hints then student can give correct response. Now generate only the best adaptive prompt so model can provide the best response for next step.
    """,
    input_variables=['response']
)

In [14]:
str_parser = StrOutputParser()

In [21]:
branch_chain = RunnableBranch(
    (lambda x: x.sentiment == 'correct', prompt_correct | model | str_parser),
    (lambda x: x.sentiment == 'partially_correct', prompt_partially_correct | model | str_parser),
    (lambda x: x.sentiment == 'incorrect', prompt_incorrect | model | str_parser),
    RunnableLambda(lambda x: "Could find any kind of sentiment.")
)

In [22]:
final_chain = classifier_chain | branch_chain

In [23]:
adaptive_prompt = final_chain.invoke(
    {
        'response': "the sum of 2+2=5"
    }
)

In [24]:
print(adaptive_prompt)

Based on the scenario provided, here is the adaptive prompt to guide the model to provide the best response for the next step:

"Given the student's incorrect response, craft a probing question or explanation that identifies the specific misconception or gap in understanding the student has. Offer additional context or a simplified example related to the topic to clarify the concept, and ask a guiding question that encourages the student to think through the problem again, leading them towards the correct understanding."


In [25]:
final_chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
      +----------+       
      | ChatGroq |       
      +----------+       
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     


## *****Access Gemma3:1B*****

In [26]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama

In [27]:
llm = Ollama(model="gemma3:1b")  # Replace "llama3" with your model name

In [28]:
llm.invoke("hi")

"Hi there! How's your day going so far? 😊 \n\nWhat can I do for you today?"

In [29]:
chat_model = ChatOllama(model="gemma3:1b") # Replace "llama3" with your model name

In [32]:
chat_model.invoke("hi sir").content

'Hi there! How can I help you today? 😊 \n\nDo you have any questions for me, or would you like to chat about something?'